In [1]:
# Import libraries
import os
import sys
import logging
import pickle # remove
import shutil

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, "..")) # absolute path for ekr directory
repo_dir = os.path.abspath(os.path.join(kit_dir, "..")) # absolute path for starter-kit directory

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')
PERSIST_DIRECTORY = os.path.join(kit_dir,f"data/my-vector-db")
#save_location = kit_dir + "/data/my-vector-db"

/Users/petrojm/miniconda3/envs/complex_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# Specify PDF file
pdf_file = kit_dir + '/data/tmp/milan_paper.pdf'

# Initialize DocumentRetrieval class
documentRetrieval =  DocumentRetrieval()
#print(documentRetrieval.loaders)

# Get pdf text
raw_text = []
meta_data = []
loader = UnstructuredPDFLoader(pdf_file)
docs_unstructured = loader.load()
for doc in docs_unstructured:
    raw_text.append(doc.page_content)
    meta_data.append({"filename": pdf_file})

#raw_text, meta_data = documentRetrieval.get_data_for_splitting(docs)
#with open(kit_dir+'/streamlit/'+'raw_text.pkl', 'rb') as file:
#    raw_text = pickle.load(file)
#with open(kit_dir+'/streamlit/'+'meta_data.pkl', 'rb') as file:
#    meta_data = pickle.load(file)

# Get the text chunks
text_chunks = documentRetrieval.get_text_chunks_with_metadata(docs=raw_text, meta_data=meta_data)
print(len(text_chunks))
#print(type(text_chunks[0])) # list of langchain_core.documents.base.Document
#print(text_chunks[0].page_content)

/Users/petrojm/miniconda3/envs/complex_rag/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2024-06-13 23:17:41,116 [ERROR] - Following dependencies are missing: pikepdf. Please install them using `pip install pikepdf`.
2024-06-13 23:17:41,119 [WARNING] - PDF text extraction failed, skip text extraction...
2024-06-13 23:17:44,050 [INFO] - Processing entire page OCR with tesseract...
2024-06-13 23:17:50,419 [INFO] - Processing entire page OCR with tesseract...
2024-06-13 23:17:58,305 [INFO] - Processing entire page OCR with tesse

15


In [3]:
# Create vector store
embeddings = documentRetrieval.load_embedding_model()
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"The directory Chroma has been deleted.")
#vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=None)
vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=PERSIST_DIRECTORY)

# Create conversation chain
documentRetrieval.init_retriever(vectorstore)
conversation = documentRetrieval.get_qa_retrieval_chain()

2024-06-13 23:18:13,804 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-06-13 23:18:21,892 [INFO] - Use pytorch device: cpu


max_seq_length  512
The directory Chroma has been deleted.


2024-06-13 23:18:22,682 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-06-13 23:18:41,436 [INFO] - Vector store saved to /Users/petrojm/Documents/projects/workshops/new/ai-starter-kit/enterprise_knowledge_retriever/data/my-vector-db


In [4]:
# Ask questions about your data
user_question = "What is DFNN-BC?"
#user_question = "What is monolithic LLMs?"

response = conversation.invoke({"question":user_question})
#print(response)
print(response['question'])
print(response['answer'])
print(len(response['source_documents']))
print('\nSource #1:')
print(response['source_documents'][0].page_content)
print(response['source_documents'][0].metadata)

print('\nSource #2:')
print(response['source_documents'][1].page_content)
print(response['source_documents'][1].metadata)

print('\nSource #3:')
print(response['source_documents'][2].page_content)
print(response['source_documents'][2].metadata)


What is DFNN-BC?
 DFNN-BC stands for Deep Feedforward Neural Network with Boundary Conditions. It is a neural network model developed for efficient evaluation of thermophysical properties in numerical simulations of complex real-fluid flows. The model incorporates boundary information and can be coupled to a flow solver in a robust manner. It is able to accelerate the evaluation of real-fluid properties and overall flowfield simulation significantly, while also reducing memory usage.
3

Source #1:
Source: /Users/petrojm/Documents/projects/workshops/new/ai-starter-kit/enterprise_knowledge_retriever/data/tmp/milan_paper, Text: 
A neural network model, referred to as DFNN-BC, is developed to achieve the desired accuracy when incorporating boundary information (i.e., the flow behavior along the bound- aries of the domain). The model can be coupled to a flow solver in a robust manner. The proposed methodology is implemented and tested in two model problems with differ- ent numerical formula